## Data

In [1]:
import torch
import torch.nn as nn
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

corpus = [
    "ăn quả nhớ kẻ trồng cây",
    "có chí thì nên"    
]
data_size = len(corpus)

# Define the max vocabulary size and sequence length
vocab_size = 14
sequence_length = 7

In [2]:
# Initialize the tokenizer and define a trainer
tokenizer = Tokenizer(WordLevel())
tokenizer.pre_tokenizer = Whitespace()
tokenizer.enable_padding(pad_id=1, 
                         pad_token="<pad>", 
                         length=sequence_length)
tokenizer.enable_truncation(max_length=sequence_length)

# Train the tokenizer on your corpus
trainer = WordLevelTrainer(vocab_size=vocab_size, 
                           special_tokens=["<unk>", "<pad>", "<sos>", "<eos>"])
tokenizer.train_from_iterator(corpus, trainer)

In [3]:
data_x = []
data_y = []

corpus[0] = '<sos> ' + corpus[0] + ' <eos>'
corpus[1] = '<sos> ' + corpus[1] + ' <eos>'

for vector in corpus:
    vector = vector.split()
    data_x.append( ' '.join(vector[:-1]) )
    data_y.append( ' '.join(vector[1:]) ) 

print(data_x)
print(data_y)

['<sos> ăn quả nhớ kẻ trồng cây', '<sos> có chí thì nên']
['ăn quả nhớ kẻ trồng cây <eos>', 'có chí thì nên <eos>']


In [4]:
# Tokenize and numericalize your samples
def vectorize(x, y, tokenizer, sequence_length):     
    x_ids = tokenizer.encode(x).ids
    y_ids = tokenizer.encode(y).ids
    print(x_ids, y_ids)
    return x_ids, y_ids

# Vectorize the samples
data_x_ids = []
data_y_ids = []
for x, y in zip(data_x, data_y):
    x_ids, y_ids = vectorize(x, y, tokenizer, sequence_length)
    data_x_ids.append(x_ids)
    data_y_ids.append(y_ids)

data_x_ids = torch.tensor(data_x_ids, dtype=torch.long)
data_y_ids = torch.tensor(data_y_ids, dtype=torch.long)

[2, 13, 10, 8, 7, 12, 5] [13, 10, 8, 7, 12, 5, 3]
[2, 6, 4, 11, 9, 1, 1] [6, 4, 11, 9, 3, 1, 1]


## Train with full data

In [5]:
class TG_Model(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, sequence_length):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim)
        self.mask = torch.triu(torch.ones(sequence_length, sequence_length), 
                               diagonal=1).bool()
        self.transformer = nn.TransformerEncoderLayer(d_model=embed_dim, 
                                                       nhead=num_heads, 
                                                       batch_first=True, 
                                                       dim_feedforward=4,
                                                       dropout=0.0)
        self.linear = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)                        # [n, seq_len, embed_dim]
        x = self.transformer(x, src_mask=self.mask)  # [n, seq_len, embed_dim]        
        x = self.linear(x)                           # [n, seq_len, vocab_size]
        return x.permute(0,2,1)                      # [n, vocab_size, seq_len]

model = TG_Model(vocab_size, 8, 1, sequence_length)
print(model)

TG_Model(
  (embedding): Embedding(14, 8)
  (transformer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
    )
    (linear1): Linear(in_features=8, out_features=4, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (linear2): Linear(in_features=4, out_features=8, bias=True)
    (norm1): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.0, inplace=False)
    (dropout2): Dropout(p=0.0, inplace=False)
  )
  (linear): Linear(in_features=8, out_features=14, bias=True)
)


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [7]:
# train
for _ in range(40):
    optimizer.zero_grad()
    outputs = model(data_x_ids)
    loss = criterion(outputs, data_y_ids)
    #print(loss.item())
    loss.backward()
    optimizer.step()

In [8]:
outputs = model(data_x_ids)
#print(outputs)
print(torch.argmax(outputs, axis=1))

tensor([[13, 10,  8,  7, 12,  5,  3],
        [13,  4, 11,  9,  3,  1,  1]])


In [9]:
data_y_ids

tensor([[13, 10,  8,  7, 12,  5,  3],
        [ 6,  4, 11,  9,  3,  1,  1]])

## Inference

In [10]:
promt = '<sos>'
promt_length = 1
promt_ids = tokenizer.encode(promt).ids
print(promt_ids)

[2, 1, 1, 1, 1, 1, 1]


In [11]:
for i in range(sequence_length - promt_length):
    promt_tensor = torch.tensor(promt_ids, dtype=torch.long).reshape(1, -1)
    outputs = model(promt_tensor)
    outputs = torch.argmax(outputs, axis=1)   
    next_id = outputs[0][promt_length+i-1]

    promt_ids[promt_length+i] = next_id.item()
print(promt_ids)

[2, 13, 10, 8, 7, 12, 5]


In [12]:
print(tokenizer.get_vocab())

{'có': 6, 'nhớ': 8, 'chí': 4, '<eos>': 3, 'cây': 5, 'kẻ': 7, '<pad>': 1, 'quả': 10, 'trồng': 12, 'nên': 9, 'thì': 11, 'ăn': 13, '<unk>': 0, '<sos>': 2}
